In [0]:
#Necessary SQL connection pieces
database = "boogaloo-capstone-human-life"
tableIR="dbo.health_insurance_rates"
tableSA="dbo.state_abbrs"
table18_S1901="dbo.S1901_2018"
table19_S1901="dbo.S1901_2019"
table18_S2701="dbo.S2701_2018"
table19_S2701="dbo.S2701_2019"
tableLE="dbo.cdc_state"
tableLE_All="dbo.Model_Data"
user = "bglcap"
password  = "48Gheq0Iz9t"
server = "gen10-data-fundamentals-22-02-sql-server.database.windows.net"

In [0]:
#Read in data from SQL Server
Insured_Rates=spark.read.format("jdbc").option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .option("dbtable", tableIR) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .load()
State_Abbrs=spark.read.format('jdbc').option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .option("dbtable", tableSA) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .load()
S1901_18_df=spark.read.format('jdbc').option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .option("dbtable", table18_S1901) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .load()
S1901_19_df=spark.read.format('jdbc').option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .option("dbtable", table19_S1901) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .load()
S2701_18_df=spark.read.format('jdbc').option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .option("dbtable", table18_S2701) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .load()
S2701_19_df=spark.read.format('jdbc').option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .option("dbtable", table19_S2701) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .load()
Life_Expectancy = spark.read.format('jdbc').option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .option("dbtable", tableLE) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .load()

In [0]:
display(S1901_18_df)

Median Household Income,NAME,state
63296,Rhode Island,44
59116,Nebraska,31
52413,North Carolina,37
51015,South Carolina,45
56499,South Dakota,46
50972,Tennessee,47
59570,Texas,48
60076,Vermont,50
57598,Nevada,32
76106,Connecticut,9


In [0]:
from pyspark.sql.functions import lit
from pyspark.sql.functions import concat_ws
#Add a Year column (all 2018), make a concatenated column of NAME and Year called State:Year, make MHI a double, reduce dataframe to just relevant columns
S1901_18_df_clean=S1901_18_df.withColumn("Year", lit(2018))
S1901_18_df_clean=S1901_18_df_clean.withColumn("State:Year",concat_ws(":","NAME",'Year'))
S1901_18_df_clean=S1901_18_df_clean[['State:Year', 'Median Household Income']]


#Same as cleaning of S1901_18_df just for different year(2019)
S1901_19_df_clean=S1901_19_df.withColumn("Year", lit(2019))
S1901_19_df_clean=S1901_19_df_clean.withColumn("State:Year",concat_ws(":","NAME",'Year'))
S1901_19_df_clean=S1901_19_df_clean[['State:Year', 'Median Household Income']]


#Change column name for future join
Life_Expectancy_clean=Life_Expectancy.withColumnRenamed("STATE","Code")
Life_Expectancy_clean=Life_Expectancy_clean.withColumnRenamed("RATE","Life Expectancy")

#Change columns from number of people to % of population, make State:Year as above, reduce DF to relevant columns
S2701_18_P_df=S2701_18_df.withColumn("Male",S2701_18_df["Male"]/S2701_18_df["Total Population"])
S2701_18_P_df=S2701_18_P_df.withColumn("Female",S2701_18_P_df["Female"]/S2701_18_P_df["Total Population"])
S2701_18_P_df=S2701_18_P_df.withColumn("White",S2701_18_P_df["White"]/S2701_18_P_df["Total Population"])
S2701_18_P_df=S2701_18_P_df.withColumn("Black or African American",S2701_18_P_df["Black or African American"]/S2701_18_P_df["Total Population"])
S2701_18_P_df=S2701_18_P_df.withColumn("American Indian and Alaska Native",S2701_18_P_df["American Indian and Alaska Native"]/S2701_18_P_df["Total Population"])
S2701_18_P_df=S2701_18_P_df.withColumn("Asian",S2701_18_P_df["Asian"]/S2701_18_P_df["Total Population"])
S2701_18_P_df=S2701_18_P_df.withColumn("Native Hawaiian and other Pacific Islander",S2701_18_P_df["Native Hawaiian and other Pacific Islander"]/S2701_18_P_df["Total Population"])
S2701_18_P_df=S2701_18_P_df.withColumn("Other",S2701_18_P_df["Other"]/S2701_18_P_df["Total Population"])
S2701_18_P_df=S2701_18_P_df.withColumn("Multiracial",S2701_18_P_df["Multiracial"]/S2701_18_P_df["Total Population"])
S2701_18_P_df=S2701_18_P_df.withColumn("Hispanic or Latino",S2701_18_P_df["Hispanic or Latino"]/S2701_18_P_df["Total Population"])
S2701_18_P_df=S2701_18_P_df.withColumn("Year", lit(2018))
S2701_18_P_df=S2701_18_P_df.withColumn("State:Year",concat_ws(":","State",'Year'))
S2701_18_P_df=S2701_18_P_df[['State:Year',"Male","Female","White","Black or African American","American Indian and Alaska Native","Asian","Native Hawaiian and other Pacific Islander","Other","Multiracial","Hispanic or Latino"]]


#Change columns from number of people to % of population, make State:Year as above, reduce DF to relevant columns
S2701_19_P_df=S2701_19_df.withColumn("Male",S2701_19_df["Male"]/S2701_19_df["Total Population"])
S2701_19_P_df=S2701_19_P_df.withColumn("Female",S2701_19_P_df["Female"]/S2701_19_P_df["Total Population"])
S2701_19_P_df=S2701_19_P_df.withColumn("White",S2701_19_P_df["White"]/S2701_19_P_df["Total Population"])
S2701_19_P_df=S2701_19_P_df.withColumn("Black or African American",S2701_19_P_df["Black or African American"]/S2701_19_P_df["Total Population"])
S2701_19_P_df=S2701_19_P_df.withColumn("American Indian and Alaska Native",S2701_19_P_df["American Indian and Alaska Native"]/S2701_19_P_df["Total Population"])
S2701_19_P_df=S2701_19_P_df.withColumn("Asian",S2701_19_P_df["Asian"]/S2701_19_P_df["Total Population"])
S2701_19_P_df=S2701_19_P_df.withColumn("Native Hawaiian and other Pacific Islander",S2701_19_P_df["Native Hawaiian and other Pacific Islander"]/S2701_19_P_df["Total Population"])
S2701_19_P_df=S2701_19_P_df.withColumn("Other",S2701_19_P_df["Other"]/S2701_19_P_df["Total Population"])
S2701_19_P_df=S2701_19_P_df.withColumn("Multiracial",S2701_19_P_df["Multiracial"]/S2701_19_P_df["Total Population"])
S2701_19_P_df=S2701_19_P_df.withColumn("Hispanic or Latino",S2701_19_P_df["Hispanic or Latino"]/S2701_19_P_df["Total Population"])
S2701_19_P_df=S2701_19_P_df.withColumn("Year", lit(2019))
S2701_19_P_df=S2701_19_P_df.withColumn("State:Year",concat_ws(":","State",'Year'))
S2701_19_P_df=S2701_19_P_df[['State:Year',"Male","Female","White","Black or African American","American Indian and Alaska Native","Asian","Native Hawaiian and other Pacific Islander","Other","Multiracial","Hispanic or Latino"]]

In [0]:
#Join Life_Expectancy_clean with State_Abbrs to get full state names, then create State:Year_LE as for previous tables and reduce to relevant columns.
LE_States=Life_Expectancy_clean.join(State_Abbrs,Life_Expectancy_clean["Code"] == State_Abbrs["Code"],"inner")
LE_States=LE_States.withColumn("State:Year_LE",concat_ws(":","State",'YEAR'))
# LE_States=LE_States.withColumn("Life Expectancy",LE_States.RATE)
LE_States=LE_States[['State:Year_LE',"Life Expectancy"]]

#Join previously created DF with Insured_Rates on the State:Year columns, change %insured to a decimal value, and reduce to relevant columns
LE_States_with_Insurance=LE_States.join(Insured_Rates,LE_States["State:Year_LE"]==Insured_Rates["State:Year"],"inner")
#LE_States_with_Insurance=LE_States_with_Insurance.withColumn("Percent Insured",LE_States_with_Insurance["Percent Insured"]/100.0)
LE_States_with_Insurance=LE_States_with_Insurance[['State:Year',"Life Expectancy","Percent Insured"]]

#combine the S1901 DFs and rename columns to avoid future join confusion
S1901_ALL_df=S1901_18_df_clean.union(S1901_19_df_clean)
S1901_ALL_df=S1901_ALL_df.withColumnRenamed("State:Year","State:Year_to_kill_first")

#combine the S2701 DFs and rename columns to avoid future join confusion
S2701_ALL_df=S2701_18_P_df.union(S2701_19_P_df)
S2701_ALL_df=S2701_ALL_df.withColumnRenamed("State:Year","State:Year_to_kill_second")

#Join S1901_ALL_df and S2701_ALL_df on the State:Year columns and reduce to relevant columns
All_Census_Tables=S1901_ALL_df.join(S2701_ALL_df,S1901_ALL_df["State:Year_to_kill_first"] == S2701_ALL_df["State:Year_to_kill_second"],"outer")
All_Census_Tables_cleaned=All_Census_Tables[['State:Year_to_kill_second',"Male","Female","White","Black or African American","American Indian and Alaska Native","Asian","Native Hawaiian and other Pacific Islander","Other","Multiracial","Hispanic or Latino","Median Household Income"]]

#Join All_Census_Tables with LE_States_with_Insurance on the State:Year columns and reduce to relevant columns
LE_All_Data=LE_States_with_Insurance.join(All_Census_Tables_cleaned,LE_States_with_Insurance["State:Year"]==All_Census_Tables_cleaned["State:Year_to_kill_second"],"inner")
LE_All_Data_cleaned=LE_All_Data[["Male","Female","White","Black or African American","American Indian and Alaska Native","Asian","Native Hawaiian and other Pacific Islander","Other","Multiracial","Hispanic or Latino","Median Household Income","Percent Insured","Life Expectancy"]]


In [0]:
display(LE_All_Data_cleaned)

Male,Female,White,Black or African American,American Indian and Alaska Native,Asian,Native Hawaiian and other Pacific Islander,Other,Multiracial,Hispanic or Latino,Median Household Income,Percent Insured,Life Expectancy
0.48752413105610454,0.5124758689438955,0.8242437597231131,0.11373628892549749,0.00434417570253205,0.01937745974476378,0.001073160643816075,0.011671082118707562,0.025554073141569867,0.04088902675673586,53560,0.9059999999999999,76.5999984741211
0.5067046570966742,0.4932953429033258,0.9159032680059052,0.00898916151945375,0.02384850994972164,0.008533836759068405,0.0010314857071806467,0.0151342945355775,0.026559443523092845,0.09816801833908034,64049,0.877,77.69999694824219
0.49326594013113484,0.5067340598688651,0.6013021231362965,0.05650936354547794,0.0074845912246790415,0.14426510844377397,0.0038995499243699504,0.13849356345178507,0.04804570027361759,0.38959583119426766,71228,0.9279999999999999,80.80000305175781
0.4873595245380416,0.5126404754619583,0.8234529984040094,0.1128242288758274,0.004355797221564251,0.019951968379990307,0.0013339149075465597,0.011726799670886419,0.02635429254017561,0.041759999813041584,55461,0.9,76.9000015258789
0.4856111235753896,0.5143888764246104,0.7650792274866802,0.1042226142804372,0.002663389694534527,0.0448782013366239,3.322863339384174E-4,0.051292772304826394,0.03153150856295934,0.15716152118587412,76106,0.9470000000000001,80.4000015258789
0.49872098096979933,0.5012790190302007,0.8409353888277674,0.04035213947226212,0.00956105473614103,0.03203774533716849,0.0015138669011701062,0.03886020741662447,0.036739597308866376,0.21506028366287266,72331,0.92,80.0
0.4948003980227843,0.5051996019772157,0.8578288270700357,0.06187684075899118,0.008536670425147637,0.027831277034162456,3.431351759820799E-4,0.020157701982718466,0.02342554755296251,0.0668694750943096,59209,0.945,79.30000305175781
0.48491211768298803,0.515087882317012,0.7557992629040586,0.15864390198315598,0.002786684403025433,0.02744911492691244,6.278505390405923E-4,0.02834889602207542,0.026344289221731494,0.25306472686579257,53267,0.87,78.9000015258789
0.4816301936390712,0.5183698063609288,0.6235429031690581,0.3186810221952956,0.005665189846484696,0.01756851552152781,2.770408787745449E-4,0.013779219258004244,0.020486109130854982,0.05087330391976651,49469,0.9109999999999999,75.69999694824219
0.5013831553657148,0.49861684463428524,0.8646865374288843,0.011618830711878072,0.010765002961694655,0.02333958670438445,0.008896155723592968,0.05039895769189062,0.030294928777674913,0.1401531223816519,71621,0.903,79.69999694824219


In [0]:
#Write cleaned and joined data for the ML model to the SQL server
LE_All_Data_cleaned.write.format('jdbc').option("url", f"jdbc:sqlserver://{server}:1433;databaseName={database};") \
    .mode("overwrite") \
    .option("dbtable", tableLE_All) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .save()